# anomaly_score_unsupervised

Send arbitrary time-series data to the component and train an unsupervised LSTM-Autoencoder model. The moment unseen patters occur the anomaly score rises.

Future work:

- reset / rollback model (for regular flushing or after a real anomaly (true positive) occurred)
- add check-pointing for service persistence and rollback

In [ ]:
import numpy as np
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation

from flask import request
from flask import Flask
from flask import send_file

import time
import json

import matplotlib.pyplot as plt
import numpy as np

import pickle
import re
import logging
import sys

In [ ]:
# COS URL where the results of the SQL job are to be stored
target_dir_path = os.environ.get('target_dir_path')

# Asset name to register for the results written by the SQL job
target_asset_name = os.environ.get('target_asset_name')

# sql statement to execute
sql = os.environ.get('sql')

# (unique) Custom Resource Name (CRN) of IBM SQL Query Service
data_engine_crn = os.environ.get('data_engine_crn')

# default: CSV - (will be generated into according STORED AS … clause in the INTO clause)
format = os.environ.get('format' , 'CSV')

# optional, list of columns to use for partitioning the results of the SQL job, will be generated into according PARTITIONED BY (<columns>) clause in the INTO clause)
partition_columns = os.environ.get('partition_columns','')

# optional, number of objects to store the results of the SQL job in, will be generated into according PARTITIONED INTO <num> OBJECTS clause in INTO clause
number_of_objects = int(os.environ.get('number_of_objects', 0))

# optional, number of rows to be stored in each result object of the SQL job, will be generated into according PARTITIONED EVERY <num> ROWS clause in INTO clause
rows_per_object = int(os.environ.get('rows_per_object', 0))

# default: False, only valid when none of the above partitioning option is specified, produces exactly one object with name specified in target_dir_path, twill be generated into sqlClient.rename_exact_result(jobid) after SQL has run.
exact_name = os.environ.get('exact_name', 'False')

# default: False - will be generated into JOBPREFIX NONE in the INTO clause. Will cause results of previous runs with same output_uri to be overwritten, because no unique sub folder will be created for the result)
no_jobid_folder = os.environ.get('no_jobid_folder', 'False')

# default: output.txt - output file name containing the CPD path of the resulting asset
data_asset = os.environ.get('data_asset','output.txt')


In [ ]:
for element in sys.argv:
    logging.warning('argv raw ' +  element)

parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))


for parameter in parameters:
    exec(parameter)
    logging.warning('Parameter: ' + parameter)


for parameter in parameters:
    exec("logging.warning('final parameter: ' + str({}))".format(parameter.split('=')[0]))
    exec("logging.warning('final parameter type: ' + str(type({})))".format(parameter.split('=')[0]))

In [ ]:
with open('./anomaly-score-unsupervised/watsoniotp.healthy.phase_aligned.pickle','rb') as file_object:
    raw_data = file_object.read()
    data_healthy = pickle.loads(raw_data, encoding='latin1')

with open('./anomaly-score-unsupervised/watsoniotp.broken.phase_aligned.pickle','rb') as file_object:
    raw_data = file_object.read()
    data_broken = pickle.loads(raw_data, encoding='latin1')

In [ ]:
data_healthy = data_healthy.reshape(3000,3)
data_broken = data_broken.reshape(3000,3)


In [ ]:
def scaleData(data):
    # normalize features
    scaler = MinMaxScaler(feature_range=(0, 1))
    return scaler.fit_transform(data)


In [ ]:
data_healthy_scaled = scaleData(data_healthy)
data_broken_scaled = scaleData(data_broken)


In [ ]:
"""
data = np.array([[0,1,2,3,4,5,6,7,8,9],[10,11,12,13,14,15,16,17,18,19], [20,21,22,23,24,25,26,27,28,29]])
data = data.T
"""

In [ ]:
def lstm_data_transform(data, num_steps=5):
    x = []
    for i in range(data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps        # if index is larger than the size of the dataset, we stop
        if end_ix >= data.shape[0]:
            break        # Get a sequence of data for x
        seq = data[i:end_ix]
        x.append(seq)
    return np.array(x)

In [ ]:
timesteps = 10
dim = 3
samples = 3000
data_healthy_scaled_reshaped = lstm_data_transform(data_healthy_scaled, num_steps=timesteps)
data_broken_scaled_reshaped = lstm_data_transform(data_broken_scaled, num_steps=timesteps)

#reshape to (300,10,3)
data_healthy_scaled_reshaped.shape


In [ ]:
loss_history = []
loss_history_total = []


class LossHistory(Callback):
    def on_train_begin(self, logs):
        loss_history = [] 

    def on_train_batch_end(self, batch, logs):
        print('Loss on_train_batch_end '+str(logs.get('loss')))
        loss_history.append(logs.get('loss'))
        loss_history_total.append(logs.get('loss'))

In [ ]:
# design network

model = Sequential()
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(LSTM(50,input_shape=(timesteps,dim),return_sequences=True))
model.add(Dense(3))
model.compile(loss='mae', optimizer='adam')

def train(data):
    model.fit(data, data, epochs=5, batch_size=32, validation_data=(data, data), verbose=0, shuffle=False,callbacks=[LossHistory()])


In [ ]:
"""
for i in range(5):

    print("----------------")
    train(data_healthy_scaled_reshaped)

print("----------------broken")
train(data_broken_scaled_reshaped)
"""

In [ ]:
def doNN(data):
    #data = scaleData(data)
    train(data)


In [ ]:
app = Flask(__name__)


In [ ]:


@app.route('/send_data', methods=['POST'])
def send_data():
    message = request.get_json()
    #message = message[1:-1] # get rid of encapsulating quotes
    #json_array = json.loads()
    data = np.asarray(message)
    print(data)
    data = lstm_data_transform(data, num_steps=timesteps)
    doNN(data)
    return json.dumps(loss_history)



In [ ]:
@app.route('/reset_model', methods=['GET'])
def reset_model():
    return "not yet implemented"

In [ ]:
@app.route('/get_loss_as_image.png', methods=['GET'])
def get_loss_as_image():
    t = np.arange(0, len(loss_history_total), 1)

    fig, ax = plt.subplots()
    ax.plot(t, loss_history_total)

    ax.set(xlabel='epochs', ylabel='loss',
           title='Reconstruction error')
    ax.grid()

    filename = "anomalies.png"
    fig.savefig(filename)
    return send_file(filename, mimetype='image/png')
    


In [ ]:
app.run(host='0.0.0.0', port=8080)
